In [21]:
'''
This notebook is to be used to convert best-fit parameter values
saved in climatools.lblnew.bestfit_params into Fortran, in a form
that can be easily inserted into the kdist_params() subroutine in
the overlap version of lblnew.f
'''

'\nThis notebook is to be used to convert best-fit parameter values\nsaved in climatools.lblnew.bestfit_params into Fortran, in a form\nthat can be easily inserted into the kdist_params() subroutine in\nthe overlap version of lblnew.f\n'

In [22]:
import os
import itertools
import importlib


from climatools.lblnew import bestfit_params as bestfit
from climatools.lblnew import setup_overlap as setup

importlib.reload(bestfit)
importlib.reload(setup)

<module 'climatools.lblnew.setup_overlap' from '/chia_cluster/home/jackyu/climatools/climatools/lblnew/setup_overlap.py'>

In [23]:
# These are the gases and spectral bands
# that have been fitted using the k-distribution method

def h2o_gasbands():
    return [('h2o', '1'), 
            ('h2o', '2'),
            ('h2o', '3a'),
            ('h2o', '3b'),
            ('h2o', '3c'),
            ('h2o', '4'),
            ('h2o', '5'),
            ('h2o', '6'),
            ('h2o', '7'), 
            ('h2o', '8'),
            ('h2o', '9')]


def co2_gasbands():
    return [('co2', '3a'), ('co2', '3b'), ('co2', '3c'), 
            ('co2', '4'), ('co2', '5'), ('co2', '9')]


def o3_gasbands():
    return [('o3', '5'), ('o3', '9')]


def n2o_gasbands():
    return [('n2o', '3a'), ('n2o', '6'), ('n2o', '7')]


def ch4_gasbands():
    return [('ch4', '6'), ('ch4', '7')]

In [24]:
def into_chunks(l, chunksize):
    return itertools.zip_longest(*(chunksize * [iter(l)]))



def vector_to_F77list(array, num_values_per_line=4, dtype=float):
    '''
    '''
    if dtype == float:
        strfmt = '{:15.6e}_r8'
    elif dtype == int:
        strfmt = '{:15d}'
    else:
        raise ValueError('dtype must be either float or int.')
    
    chunks = into_chunks(array, num_values_per_line)
    
    chunks = list(chunks)
    
    lines = []
    for chunk in chunks[:-1]:
        vs = [strfmt.format(v) for v in chunk if v != None]
        line = ','.join(vs)
        line = line + ','
        lines.append(line)
        
    vs = [strfmt.format(v) for v in chunks[-1] if v != None]
    line = ','.join(vs)
    lines.append(line)
    
    return lines


def vector_to_F77(array=None, num_values_per_line=None, dtype=None):
    lines = vector_to_F77list(array=array, 
                              num_values_per_line=num_values_per_line,
                              dtype=dtype)
    
    rlines = [5 * ' ' + '&' + l for l in lines]
    
    fortran = '\n'.join(rlines)
    return fortran

In [25]:
def if_conditional(param, first=False):
    vmin, vmax = param['vmin'], param['vmax']
    
    s_if = "if (vstar == {vmin}_r8 .and. vend == {vmax}_r8) then"
    
    s_elif = ("else if (vstar == {vmin}_r8 "
              ".and. vend == {vmax}_r8) then")
    
    if first:
        s = s_if
    else:
        s = s_elif
        
    return s.format(vmin=vmin, vmax=vmax)


def comment_header(param):
    s = "! {} band{}"
    return s.format(param['molecule'], param['band'])


def nref(param):
    s = "nref = {:d}"
    return s.format(len(param['ng_refs']))


def ng(param):
    s = "ng = {:d}"
    return s.format(sum(param['ng_refs']))


def ng_refs(param):
    s = vector_to_F77(param['ng_refs'], 
                      num_values_per_line=3, dtype=int)
    ls = ['ng_refs(1:nref) = (/',
          s,
          5 * ' ' + '&' + '/)']
    return '\n'.join(ls)


def ng_adju(param):
    s = vector_to_F77(param['ng_adju'], 
                      num_values_per_line=3, dtype=int)
    ls = ['ng_adju(1:nref) = (/',
          s,
          5 * ' ' + '&' + '/)']
    return '\n'.join(ls)


def p_refs(param):
    ps, ts = zip(*param['ref_pts'])
    s = vector_to_F77(ps, 
                      num_values_per_line=3, dtype=float)
    ls = ['p_refs(1:nref) = (/',
          s,
          5 * ' ' + '&' + '/)']
    return '\n'.join(ls)    


def t_refs(param):
    ps, ts = zip(*param['ref_pts'])
    s = vector_to_F77(ts, 
                      num_values_per_line=3, dtype=float)
    ls = ['t_refs(1:nref) = (/',
          s,
          5 * ' ' + '&' + '/)']
    return '\n'.join(ls)


def wgt(param):
    vs = [v for ref in param['wgt'] for v in ref]
    s = vector_to_F77(vs, 
                      num_values_per_line=3, dtype=float)
    ls = ['wgt(1:ng) = (/',
          s,
          5 * ' ' + '&' + '/)']
    return '\n'.join(ls)


def w_diffuse(param):
    vs = [v for ref in param['w_diffuse'] for v in ref]
    s = vector_to_F77(vs, 
                      num_values_per_line=3, dtype=float)
    ls = ['w_diffuse(1:ng) = (/',
          s,
          5 * ' ' + '&' + '/)']
    return '\n'.join(ls)


def option_klin(param):
    s = 'option_klin = {:d}'
    v = 0 if param['klin'] == 0 else 1
    return s.format(v)


def klin(param):
    s = 'klin = {:15.6e}_r8'
    # It doesn't matter but let's set klin to 0 when option_klin = 0
    v = 0 if param['klin'] == 0 else param['klin']
    return s.format(v)

In [26]:
def gasband_str_funcs():
    return (comment_header,
            nref,
            ng,
            ng_refs,
            ng_adju,
            p_refs,
            t_refs,
            wgt,
            w_diffuse,
            option_klin,
            klin)


In [27]:
def kdist_param_gasband(param):
    '''
    Returns list of strings for some gas and band.
    '''
    print(param['molecule'], param['band'])
    return [f(param) for f in gasband_str_funcs()]

    
def kdist_param_gas(params):
    '''
    Returns list of strings for some gas.
    '''
    molecules = [param['molecule'] for param in params]
    try:
        assert all([molecule == molecules[0] for molecule in molecules])
    except AssertionError:
        raise('All input param dicts should be for the same gas.')
        
    ls_gas = []
    for i, param in enumerate(params):
        first = True if i == 0 else False
        s_ifcond = if_conditional(param, first=first)
                 
        ls = kdist_param_gasband(param)
        ls = [3 * ' ' + l for l in ls]
        ls = [s_ifcond] + ls
        
        ls_gas.extend(ls)
       
    s = "write (*, *) 'Unrecognised spectral band for {}.'"
    s = s.format(molecules[0].upper())
    ls_else = []
    ls_else.append(s)
    ls_else.append('stop')
    ls_else = [3 * ' ' + l for l in ls_else]
    
    ls_gas.append('else')
    ls_gas.extend(ls_else)
    ls_gas.append('end if')
        
    return ls_gas


def gas2mid(gas):
    d = {'h2o': 1, 'co2': 2, 'o3': 3, 'n2o': 4, 'ch4': 5, 'o2':6}
    return d[gas]


def kdist_param():
    'Returns list of strings covering all gases and their bands'
    gasband_gs = [h2o_gasbands(), co2_gasbands(), o3_gasbands(),
                  n2o_gasbands(), ch4_gasbands()]
    
    lines = []
    for i, gasbands in enumerate(gasband_gs):
        
        params = [bestfit.kdist_params(molecule=gas, band=band)
                  for gas, band in gasbands]
        
        gas = params[0]['molecule']
        mid = gas2mid(gas)
        
        if i == 0:
            s_if = 'if (mid == {}) then'
        else:
            s_if = 'else if (mid == {}) then'
        s_if = s_if.format(mid)
        
        ls = kdist_param_gas(params)
        ls = [3 * ' ' + l for l in ls]
        ls = [s_if] + ls
        
        lines.extend(ls)
    
    s = "write (*, *) 'Unrecognised gas id{}.'"
    s = s.format(mid)
    ls_else = []
    ls_else.append(s)
    ls_else.append('stop')
    ls_else = [3 * ' ' + l for l in ls_else]
    
    lines.append('else')
    lines.extend(ls_else)
    lines.append('end if')
        
    return lines        
        

def subroutine():
    
    hs = ('subroutine kdist_params(mid, vstar, nband, nv, dv,',
          5 * ' ' + '&     nref, ng, ng_refs, ng_adju, p_refs, t_refs,',
          5 * ' ' + '&     wgt, w_diffuse, option_klin, klin)')
    hs = '\n'.join(hs)
    ls = (hs,
          '',
          'implicit none',
          'integer, parameter :: r8 = kind(1.0d0)',
          'integer, parameter :: maxnref = 5',
          'integer, parameter :: maxng = 30',
          '',
          '! Input parameters',
          'integer, intent(in) :: mid',
          'real (kind=8), intent(in) :: vstar',
          'integer, intent(in) :: nband',
          'integer, intent(in) :: nv',
          'real (kind=8), intent(in) ::dv',
          '',
          '! Output parameters',
          'integer, intent(out) :: nref',
          'integer, intent(out) :: ng',
          'integer, dimension(maxnref), intent(out) :: ng_refs',
          'integer, dimension(maxnref), intent(out) :: ng_adju',
          'real (kind=8), dimension(maxnref), intent(out) :: p_refs',
          'real (kind=8), dimension(maxnref), intent(out) :: t_refs',
          'real (kind=8), dimension(maxng), intent(out) :: wgt',
          'real (kind=8), dimension(maxng), intent(out) :: w_diffuse',
          'integer, intent(out) :: option_klin',
          'real (kind=8), intent(out) :: klin',
          '',
          '! Internal variables',
          'real (kind=8) :: vend',
          '',
          '',
          '',
          'vend = vstar + dv * nv * nband',
          '')
    
    lines = list(ls)
    lines = lines + kdist_param()
    lines.append('return')
    lines.append('end')
    return lines


def file_content():
    lines = subroutine()
    lines = [6 * ' ' + l for l in lines]
    s = '\n'.join(lines)
    return s


def script():
    s = file_content()
    
    fpath = os.path.join(setup.DIR_SRC, 'kdist_params.f')
    with open(fpath, mode='w', encoding='utf-8') as f:
        f.write(s)
        


In [28]:
script()

h2o 1
h2o 2
h2o 3a
h2o 3b
h2o 3c
h2o 4
h2o 5
h2o 6
h2o 7
h2o 8
h2o 9
co2 3a
co2 3b
co2 3c
co2 4
co2 5
co2 9
o3 5
o3 9
n2o 3a
n2o 6
n2o 7
ch4 6
ch4 7


In [29]:
os.listdir(setup.DIR_SRC)

['lblnew-overlap.f', 'kdist_params.f', 'lblcom.f']

In [30]:
setup.DIR_SRC

'/chia_cluster/home/jackyu/radiation/crd/LW/src/lblnew_-_overlap'